<a href="https://colab.research.google.com/github/AnnieCRY/DS5230-project/blob/master/data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### original data available [here](https://www.kaggle.com/chrisfilo/to-bee-or-no-to-bee/data), and downloaded into google drive already

### if you haven't download the data, please download first and save in your google drive under folder named 5230Project

In [ ]:
# load data from google drive
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
# move zip file to colab vm
! cp gdrive/My\ Drive/5230Project/501224_928375_bundle_archive.zip ./

CPU times: user 208 ms, sys: 25.8 ms, total: 234 ms
Wall time: 1min 21s


In [ ]:
%%capture
# unzip the file
! unzip 501224_928375_bundle_archive.zip

! mkdir labFiles
! mkdir audioFiles
! mv *.mp3 audioFiles/
! mv *.wav audioFiles/
! mv *.lab labFiles/

In [ ]:
# download the supporting files from github
! rm -rf Audio_based_identification_beehive_states/
! git clone https://github.com/Tonyz4516/Audio_based_identification_beehive_states.git

Cloning into 'Audio_based_identification_beehive_states'...
remote: Enumerating objects: 114, done.
remote: Total 114 (delta 0), reused 0 (delta 0), pack-reused 114
Receiving objects: 100% (114/114), 56.94 KiB | 1.42 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [ ]:
! mv Audio_based_identification_beehive_states/Bee_NotBee_classification/* ./

In [ ]:
%%capture
! pip install muda

In [ ]:
import os
from info import i, printb, printr, printp, print
from utils import load_audioFiles_saves_segments, write_Statelabels_from_beeNotBeelabels, get_uniqueHives_names_from_File, split_samples_byHive, get_samples_id_perSet, get_features_from_samples, get_GT_labels_fromFiles, labels2binary, write_sample_ids_perHive, split_samples_ramdom

In [ ]:
block_size=60 # blocks of 60 seconds
thresholds=[0, 5]  # minimum length for nobee intervals: 0 or 5 seconds (creates one label file per threshold value)
path_audioFiles="audioFiles/"  # path to audio files
annotations_path="labFiles/" # path to .lab files
path_save_audio_labels='dataset_BeeNoBee_'+str(block_size)+'sec/'  # path where to save audio segments and labels files.

In [ ]:
if not os.path.exists(path_save_audio_labels):
    os.makedirs(path_save_audio_labels)

In [ ]:
# segments audio files, assigns label BeeNotBee to each block, writes labels to csv , saves segmened blocks in wav.
# independently of flag save_audioSegments, if .wav with same name already exists it won't save again.
# new labels are just appended to existing labels file, if purpose is to redo the whole file delete before running.
load_audioFiles_saves_segments( list(os.listdir("audioFiles")), path_audioFiles, path_save_audio_labels, block_size , thresholds, annotations_path, read_beeNotBee_annotations='yes', save_audioSegments='yes')

[2020-07-16 22:21:01 RAM8.3% 0.26GB] Number of audiofiles in folder: 56
[2020-07-16 22:21:01 RAM8.3% 0.26GB] 

[2020-07-16 22:21:01 RAM8.3% 0.26GB] Processing Hive1_12_06_2018_QueenBee_H1_audio___16_40_00.wav          :::file number:  1 --------->of 56
[2020-07-16 22:21:05 RAM8.4% 0.3GB] -----------------Reading segment 0
[2020-07-16 22:21:05 RAM8.4% 0.3GB] Hive1_12_06_2018_QueenBee_H1_audio___16_40_00__segment0
[2020-07-16 22:21:05 RAM8.4% 0.3GB] ---------------------Will read BeeNotbee anotations and create labels for segment0
[2020-07-16 22:21:05 RAM8.4% 0.3GB] labFiles//Hive1_12_06_2018_QueenBee_H1_audio___16_40_00.lab
[2020-07-16 22:21:05 RAM8.4% 0.3GB] -----------------Wrote label for th 0 seconds of segment0
[2020-07-16 22:21:05 RAM8.4% 0.3GB] labFiles//Hive1_12_06_2018_QueenBee_H1_audio___16_40_00.lab
[2020-07-16 22:21:05 RAM8.4% 0.3GB] -----------------Wrote label for th 5 seconds of segment0
[2020-07-16 22:21:05 RAM8.4% 0.3GB] -----------------Saved wav file for segment 0
[20

In [ ]:
path_beeNotbee_labels=path_save_audio_labels + 'labels_BeeNotBee_th' + str(thresholds[0])+'.csv' 
# reads labels beeNotBee files and creates corresponding states label file.
write_Statelabels_from_beeNotBeelabels(path_save_audio_labels, path_beeNotbee_labels, states=['active','missing queen','swarm' ])

In [ ]:
import utils
sample_ids = utils.get_list_samples_names(path_save_audio_labels) # get sample ids from audio segments folder.

In [ ]:
# split data by Hive 
hives=write_sample_ids_perHive(sample_ids , path_save_audio_labels)  # retrieves unique hives names and also writes these to a file
#hives=get_uniqueHives_names_from_File(path_save_audio_labels)
for i in range(3):
    split_dict = split_samples_byHive(0.1, 0.5, hives, path_save_audio_labels+'split_byHive_'+str(i))

[2020-07-16 22:50:18 RAM8.4% 0.35GB] hives for testing: ['CF001']
[2020-07-16 22:50:18 RAM8.4% 0.35GB] hives for training: ['CF003', 'GH001', 'Hive1']
[2020-07-16 22:50:18 RAM8.4% 0.35GB] hives for validation: ['CJ001', 'Hive3']
[2020-07-16 22:50:18 RAM8.4% 0.35GB] hives for testing: ['Hive1']
[2020-07-16 22:50:18 RAM8.4% 0.35GB] hives for training: ['CF003', 'CJ001', 'Hive3']
[2020-07-16 22:50:18 RAM8.4% 0.35GB] hives for validation: ['GH001', 'CF001']
[2020-07-16 22:50:18 RAM8.4% 0.35GB] hives for testing: ['Hive3']
[2020-07-16 22:50:18 RAM8.4% 0.35GB] hives for training: ['CF003', 'GH001', 'Hive1']
[2020-07-16 22:50:18 RAM8.4% 0.35GB] hives for validation: ['CF001', 'CJ001']


In [ ]:
#split data randomly
for i in range(3):
    split_dict = split_samples_ramdom(0.1,0.5,path_save_audio_labels, path_save_audio_labels+'split_random_'+str(i))

[2020-07-16 22:50:18 RAM8.4% 0.35GB] samples for testing: ['Hive3_12_07_2017_NO_QueenBee_H3_audio___15_40_00__segment4.wav', 'Hive3_12_07_2017_NO_QueenBee_H3_audio___17_00_00__segment0.wav', 'Hive3_20_07_2017_QueenBee_H3_audio___06_20_00__segment2.wav', 'Hive3_12_07_2017_NO_QueenBee_H3_audio___15_30_00__segment4.wav', 'GH001 - Active - Day - 141022_0659_0751__segment30.wav', 'Hive1_31_05_2018_NO_QueenBee_H1_audio___15_20_00__segment6.wav', 'CF003 - Active - Day - (219)__segment1.wav', 'Hive1_12_06_2018_QueenBee_H1_audio___15_10_00__segment7.wav', 'Hive1_31_05_2018_NO_QueenBee_H1_audio___15_30_00__segment6.wav', 'Hive1_12_06_2018_QueenBee_H1_audio___15_30_00__segment3.wav', 'CF003 - Active - Day - (218)__segment4.wav', 'Hive1_12_06_2018_QueenBee_H1_audio___16_40_00__segment6.wav', 'CF003 - Active - Day - (221)__segment3.wav', 'Hive3_15_07_2017_NO_QueenBee_H3_audio___06_40_00__segment5.wav', 'Hive3_15_07_2017_NO_QueenBee_H3_audio___06_20_00__segment7.wav', 'Hive3_12_07_2017_NO_QueenBee_H